In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using Distributions
using BenchmarkTools
using Random
using Plots

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1278
┌ Warning: Package GraphPPL does not have ReactiveMP in its dependencies:
│ - If you have GraphPPL checked out for development and have
│   added ReactiveMP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with GraphPPL
└ Loading ReactiveMP into GraphPPL from project dependency, future 

In [2]:
@model function linear_gaussian_ssm(n, x1_prior, A, B, P, Q)
     
    x = randomvar(n) # Create a sequence of random variables
    y = datavar(Vector{Float64}, n) # Create a sequence of observed data points
    
    x[1] ~ MvGaussianMeanCovariance(mean(x1_prior), cov(x1_prior)) # Set a prior distribution for x_1
    y[1] ~ MvGaussianMeanCovariance(B * x[1], Q)
    
    for t in 2:n
        x[t] ~ MvGaussianMeanCovariance(A * x[t - 1], P)
        y[t] ~ MvGaussianMeanCovariance(B * x[t], Q)    
    end
    
    return x, y
end

linear_gaussian_ssm (generic function with 1 method)

In [3]:
function generate_data(; seed, n, A, B, P, Q)
    Random.seed!(seed)

    x_prev = zeros(2)

    x = Vector{Vector{Float64}}(undef, n)
    y = Vector{Vector{Float64}}(undef, n)

    for i in 1:n
        x[i] = rand(MvNormal(A * x_prev, P))
        y[i] = rand(MvNormal(B * x[i], Q))

        x_prev = x[i]
    end
   
    return y
end



generate_data (generic function with 1 method)

In [4]:
seed = 1234
n = 10_000
θ = π / 100
A = [ cos(θ) -sin(θ); sin(θ) cos(θ) ]
B = [ 1.3 0.0; 0.0 0.7 ]
P = [ 0.1 0.0; 0.0 0.1 ]
Q = [ 1.0 0.0; 0.0 1.0 ]

fake_data = generate_data(
    seed = seed,
    n    = n,
    A    = A,
    B    = B,
    P    = P,
    Q    = Q,
);

In [5]:
function inference(observations; A, B, P, Q)
    n = length(observations) 
    
    x1_prior = MvGaussianMeanCovariance([ 0.0, 0.0 ], [ 100.0 0.0; 0.0 100.0 ])
    
    break_stackoverflow = PostponeScheduler()
    
    model, (x, y) = linear_gaussian_ssm(n, x1_prior, A, B, P, Q, options = (outbound_message_portal = ScheduleOnPortal(break_stackoverflow), ))
    
    xbuffer = Vector{Marginal}(undef, n)
    
    xsub = subscribe!(collectLatest(getmarginals(x)), (ms) -> copyto!(xbuffer, ms))
    
    wait(break_stackoverflow)
    
    update!(y, observations)
    
    wait(break_stackoverflow)
    
    unsubscribe!(xsub)
    
    wait(break_stackoverflow)
    
    return xbuffer
end

inference (generic function with 1 method)

In [6]:
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =     100, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =     200, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =     500, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =     700, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =   1_000, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =   2_000, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =   5_000, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =   7_000, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =  10_000, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =  20_000, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n =  50_000, A = A, B = B, P = P, Q = Q))
@btime inference(data, A = $A, B = $B, P = $P, Q = $Q) setup=(data=generate_data(seed = seed, n = 100_000, A = A, B = B, P = P, Q = Q))

;

  9.459 ms (131884 allocations: 8.47 MiB)
  21.023 ms (264192 allocations: 16.99 MiB)
  59.845 ms (661101 allocations: 42.48 MiB)
  97.034 ms (925720 allocations: 59.55 MiB)
  140.228 ms (1322626 allocations: 85.01 MiB)
  230.512 ms (2645664 allocations: 170.07 MiB)
  894.201 ms (6614780 allocations: 425.60 MiB)
  1.343 s (9260852 allocations: 595.46 MiB)
  1.926 s (13229956 allocations: 851.23 MiB)
  3.837 s (26460313 allocations: 1.66 GiB)
  14.370 s (66151365 allocations: 4.15 GiB)
  49.559 s (132303125 allocations: 8.30 GiB)


In [7]:
import Pkg; Pkg.status(); versioninfo()

Status `~/Projects/LaTeX/RMP_Experiments/Project.toml`
  [6e4b80f9] BenchmarkTools v0.5.0
  [31c24e10] Distributions v0.24.12
  [b3f8163a] GraphPPL v0.1.0 `~/.julia/dev/GraphPPL.jl`
  [91a5bcdd] Plots v1.10.1
  [a194aa59] ReactiveMP v0.5.0 `~/.julia/dev/ReactiveMP`
  [df971d30] Rocket v1.3.3 `~/.julia/dev/Rocket`
Julia Version 1.5.3
Commit 788b2c77c1 (2020-11-09 13:37 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)
